In [43]:
import time
import matplotlib.pyplot as plt

In [62]:
count_freq = counterlogic._count_frequency

powers = np.linspace(10e-3, 100e-3, 10)
counts = np.zeros(len(powers))

for idx, power in enumerate(powers):
    laserlogic.set_power(power)
    
    # Wait for laser to settle
    time.sleep(3)
        
    # Sleep for 10s
    time.sleep(300 / count_freq)
    
    counts[idx] = np.mean(counterlogic.countdata_smoothed[0, -150:])
    print(f"Power={power:.2f}, counts={counts[idx]:.2f}")

Power=0.01, counts=2559.17
Power=0.02, counts=598.67
Power=0.03, counts=12356.83
Power=0.04, counts=13756.67
Power=0.05, counts=15739.00
Power=0.06, counts=19485.50
Power=0.07, counts=28432.67
Power=0.08, counts=28826.33
Power=0.09, counts=34143.17
Power=0.10, counts=37751.67


In [61]:
fig, ax = plt.subplots(figsize=(4, 3))

ax.plot(powers, counts, ".-")
ax.set_xlabel("Power (in W)")
ax.set_ylabel("Mean counts (kcps)")
fig.tight_layout()

plt.savefig("FR0612-3G6.png", dpi=300)